## Google Model Viewer

Google has developed a `model-viewer` web component for interactively viewing very large and detailed 3D models.

In this notebook we will demonstrate how to create a **Panel** `ModelViewer` component that will enable you to use it in your awesome app. Powered by **Python**.

[![ModelViewer Image](https://codelabs.developers.google.com/codelabs/model-viewer/img/86a799664535dbcc.gif)](https://modelviewer.dev/)

**Author:** [Marc Skov Madsen](datamodelsanalytics.com) ([awesome-panel.org](https://awesomepanel.org))

**Resources:**
[modelviewer.dev](https://modelviewer.dev/). 
[examples](https://modelviewer.dev/examples/tester.html)
[codelabs](https://codelabs.developers.google.com/codelabs/model-viewer/index.html?index=..%2F..index#0)
[Github](https://github.com/google/model-viewer/tree/master/packages/model-viewer)

**Tags:** 
[model-viewer](https://modelviewer.dev/)
[Panel](https://panel.holoviz.org/index.html)
[Python](https://www.python.org/)

**License:**
[MIT](https://opensource.org/licenses/MIT)

## Code

### HTML

The `model-viewer` can be used by importing the *javascript* packages below.

In [ ]:
%%HTML

<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js"></script>
<script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.js"></script>
<script nomodule src="https://unpkg.com/@google/model-viewer/dist/model-viewer-legacy.js"></script>
<script src="https://unpkg.com/resize-observer-polyfill@1.5.1/dist/ResizeObserver.js"></script>

<model-viewer src="https://modelviewer.dev/shared-assets/models/Astronaut.glb" alt="A 3D model of an astronaut"
auto-rotate camera-controls style="height:200px;width:200px;">
</model-viewer>

## `ModelViewer` Implementation

In [ ]:
import panel as pn
import param

pn.extension()

In [ ]:
JS = """
<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js"></script>
<script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.js"></script>
<script nomodule src="https://unpkg.com/@google/model-viewer/dist/model-viewer-legacy.js"></script>
<script src="https://unpkg.com/resize-observer-polyfill@1.5.1/dist/ResizeObserver.js"></script>
"""

HTML="""
<model-viewer src="https://modelviewer.dev/shared-assets/models/Astronaut.glb" alt="A 3D model of an astronaut"
auto-rotate camera-controls>
</model-viewer>
"""

MODELS = {
    "Astronaut": "https://modelviewer.dev/shared-assets/models/Astronaut.glb",
    "Boom Box": "https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/BoomBox/glTF-Binary/BoomBox.glb",
    "Brain Stem": "https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/BrainStem/glTF-Binary/BrainStem.glb",
    "Corset": "https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/Corset/glTF-Binary/Corset.glb",
    "Damaged Helmet": "https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/DamagedHelmet/glTF-Binary/DamagedHelmet.glb",
    "Flight Helmet": "https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/FlightHelmet/glTF/FlightHelmet.gltf",
    "Lantern": "https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/Lantern/glTF-Binary/Lantern.glb",
    "Monkey": "https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/Suzanne/glTF/Suzanne.gltf",
    "Water Bottles": "https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/SpecGlossVsMetalRough/glTF-Binary/SpecGlossVsMetalRough.glb",
    "Robot Expressive": "https://modelviewer.dev/shared-assets/models/RobotExpressive.glb",
    "Transparency Test": "https://modelviewer.dev/shared-assets/models/alpha-blend-litmus.glb",
    "Metal Rough Spheres": "https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/MetalRoughSpheres/glTF/MetalRoughSpheres.gltf",
}

SRC_DEFAULT = MODELS["Flight Helmet"]

HEIGHT_DEFAULT = 600
HEIGHT_BOUNDS = (50,1000)
WIDTH_DEFAULT = 600
WIDTH_BOUNDS = (50,1000)

BACKGROUND="#9E9E9E"

PARAMETERS = [
    "src",
    "height",
    "width",
    "exposure",
    # "auto_rotate",
    # "camera_controls",
]

In [ ]:
class ModelViewer(pn.pane.WebComponent):
    """A Wired ModelViewer"""
    html = param.String(HTML)
    attributes_to_watch= param.Dict({"src": "src"})
    properties_to_watch= param.Dict({
        "exposure": "exposure", 
        "auto-rotate": "auto_rotate",
        "camera-controls": "camera_controls",
    })

    src = param.ObjectSelector(default=SRC_DEFAULT, objects=MODELS)
    exposure = param.Number(1.0, bounds=(0, 2))
    auto_rotate = param.Boolean()
    camera_controls = param.Boolean()
    
    height = param.Integer(default=HEIGHT_DEFAULT, bounds=HEIGHT_BOUNDS)
    width = param.Integer(default=WIDTH_DEFAULT, bounds=WIDTH_BOUNDS)
    
    style = param.String()
    
    def __init__(self, **params):
        super().__init__(**params)
        
        self.css_pane = pn.pane.HTML()
        self.js_pane = pn.pane.HTML(JS)
        
        self._update_height_and_width()
    
    def view(self):
        return pn.Column(
            self,
            self.js_pane,
            self.css_pane,
            sizing_mode="stretch_both",
        )
    
    @param.depends("height", "width", watch=True)
    def _update_height_and_width(self):
        if self.height:
            height=self.height
        else:
            height=HEIGHT_DEFAULT
        if self.width:
            width=self.width
        else:
            width=WIDTH_DEFAULT
        
        self.css_pane.object = f"""
<style>
model-viewer {{
    height:{height}px;
    width:{width}px;
}}
</style>
"""

## Output

### Simple App

In [ ]:
simple_viewer = ModelViewer(height=200,width=200)
simple_view = simple_viewer.view()
simple_view

### ModelViewer App

In [ ]:
MODELVIEWER_LOGO = '<img src="https://avatars1.githubusercontent.com/u/1342004?v=4&amp;s=40" style="height:50px"></img>'
PANEL_LOGO = '<img src="https://panel.holoviz.org/_static/logo_stacked.png" style="height:50px"></img>'
BLUE = "#5dbcd2"
GRAY="#eeeeee"

def create_app(**params):
    top_app_bar = pn.Column(
        pn.layout.Row(
            pn.pane.HTML(MODELVIEWER_LOGO, width=50),
            pn.pane.Markdown("#### model-viewer", width=120, margin=(10,5,0,0)),
            pn.pane.HTML(PANEL_LOGO, width=75),
            sizing_mode="stretch_width"),
        pn.layout.HSpacer(height=2),
        height=60,
        sizing_mode="stretch_width",
        background=GRAY,
    )
    
    model_viewer = ModelViewer(height=500,width=650)
    
    settings_bar = pn.Param(
        model_viewer,
        parameters=["src", "height", "width", "exposure", "auto_rotate", "camera_controls"],
        width=200,
        sizing_mode="stretch_height",
        background=GRAY,
    )
    
    return pn.Column(
        top_app_bar,
        pn.Row(model_viewer,pn.layout.HSpacer(), settings_bar),
        model_viewer.css_pane,
        model_viewer.js_pane,
        background=BLUE,
        sizing_mode="stretch_width",
    )
    
app = create_app()
app

In [ ]:
def show(_):
    app = create_app()
    app.show()
    
show_button = pn.widgets.Button(name="Run on Production Server", button_type="success", sizing_mode="stretch_width")
show_button.on_click(show)
show_button